vectorized env

In [ ]:
# class MultiTaskSimpleTelEnv(SimpleTelEnv):
#     def __init__(self):
#         super().__init__()

#     def set_mu(self, new_mu: float):
#         self.mu = new_mu

# env = gym.make("gymnasium_env/SimpleTel-v0", Nf=Nf, target_sequence=true_sequence, nv_max=nv_max)
# vec_env = make_vec_env(MultiTaskSimpleTelEnv)
# vec_env.env_method.has_attr("mu")
# vec_env.env_method("set_wrapper_attr", "mu", .1)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random

import gymnasium as gym
from gymnasium.spaces import Dict, Box, Discrete

from stable_baselines3 import DQN, PPO, A2C
# from sb3_contrib import TRPO, CrossQ, ARS, QRDQN, TQC
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

import optuna

import os
import time
import pickle
import sys
sys.path.append('../src/')
from environments import SimpleTelEnv
from utils import save_results, load_results

In [6]:
env_name = 'SimpleTel-v0'

OUTDIR = f'results/{env_name}/'
if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)
    
SEED = 20

In [7]:
Nf = 20
nv_max = 3
random.seed(2)
true_sequence = np.array([np.full(nv_max, i) for i in range(Nf)]).flatten()
true_sequence = true_sequence.tolist()
random.shuffle(true_sequence)
print(true_sequence)

[16, 10, 13, 3, 5, 19, 2, 6, 12, 10, 14, 8, 13, 9, 2, 1, 6, 11, 7, 0, 7, 2, 8, 11, 19, 16, 15, 4, 16, 4, 3, 0, 14, 5, 15, 9, 11, 19, 10, 8, 13, 9, 17, 12, 0, 17, 4, 12, 5, 6, 14, 15, 3, 17, 7, 18, 1, 1, 18, 18]


In [10]:
# Register the environment so we can create it with gym.make()
gym.register(
    id=f"gymnasium_env/{env_name}",
    entry_point=SimpleTelEnv,
    max_episode_steps=300,  # Prevent infinite episodes. Here just set to 300 even though episode will terminate when stepping to last element of sequence
)

env = gym.make(f"gymnasium_env/{env_name}", Nf=Nf, target_sequence=true_sequence, nv_max=nv_max, off_by_lim=3)

from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env.unwrapped)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!


/home/hurra/.conda/envs/cosmoML/lib/python3.11/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment gymnasium_env/SimpleTel-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [19]:
env.action_space.n

np.int64(20)

In [11]:
from collections import defaultdict
import gymnasium as gym
import numpy as np


class QLearningAgent:
    def __init__(
        self,
        env: gym.Env,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """Initialize a Q-Learning agent.

        Args:
            env: The training environment
            learning_rate: How quickly to update Q-values (0-1)
            initial_epsilon: Starting exploration rate (usually 1.0)
            epsilon_decay: How much to reduce epsilon each episode
            final_epsilon: Minimum exploration rate (usually 0.1)
            discount_factor: How much to value future rewards (0-1)
        """
        self.env = env

        # Q-table: maps (state, action) to expected reward
        # defaultdict automatically creates entries with zeros for new states
        self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))

        self.lr = learning_rate
        self.discount_factor = discount_factor  # How much we care about future rewards

        # Exploration parameters
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        # Track learning progress
        self.training_error = []

    def get_action(self, obs: tuple[int, int, bool]) -> int:
        """Choose an action using epsilon-greedy strategy.

        Returns:
            action: 0 (stand) or 1 (hit)
        """
        # With probability epsilon: explore (random action)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()

        # With probability (1-epsilon): exploit (best known action)
        else:
            return int(np.argmax(self.q_values[obs]))

    def update(
        self,
        obs: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        next_obs: tuple[int, int, bool],
    ):
        """Update Q-value based on experience.

        This is the heart of Q-learning: learn from (state, action, reward, next_state)
        """
        # What's the best we could do from the next state?
        # (Zero if episode terminated - no future rewards possible)
        future_q_value = (not terminated) * np.max(self.q_values[next_obs])

        # What should the Q-value be? (Bellman equation)
        target = reward + self.discount_factor * future_q_value

        # How wrong was our current estimate?
        temporal_difference = target - self.q_values[obs][action]

        # Update our estimate in the direction of the error
        # Learning rate controls how big steps we take
        self.q_values[obs][action] = (
            self.q_values[obs][action] + self.lr * temporal_difference
        )

        # Track learning progress (useful for debugging)
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        """Reduce exploration rate after each episode."""
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

In [17]:
env.reset()

({'field_id': 16,
  'nvisits': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        dtype=int32),
  'index': 0},
 {'chosen_field_id': None, 'correct': None})

In [18]:
env.observation_space.sample()

{'field_id': np.int64(12),
 'index': np.int64(41),
 'nvisits': array([4, 2, 0, 4, 3, 2, 2, 2, 3, 1, 4, 2, 1, 4, 0, 2, 1, 4, 2, 3],
       dtype=int32)}

In [14]:
agent.update()

TypeError: QLearningAgent.update() missing 5 required positional arguments: 'obs', 'action', 'reward', 'terminated', and 'next_obs'

In [12]:
# Training hyperparameters
learning_rate = 0.01        # How fast to learn (higher = faster but less stable)
n_episodes = 100_000        # Number of hands to practice
start_epsilon = 1.0         # Start with 100% random actions
epsilon_decay = start_epsilon / (n_episodes / 2)  # Reduce exploration over time
final_epsilon = 0.1         # Always keep some exploration

agent = QLearningAgent(
    env=env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)


In [13]:
from tqdm import tqdm  # Progress bar

for episode in tqdm(range(n_episodes)):
    # Start a new hand
    obs, info = env.reset()
    done = False

    # Play one complete hand
    while not done:
        # Agent chooses action (initially random, gradually more intelligent)
        action = agent.get_action(obs)

        # Take action and observe result
        next_obs, reward, terminated, truncated, info = env.step(action)

        # Learn from this experience
        agent.update(obs, action, reward, terminated, next_obs)

        # Move to next state
        done = terminated or truncated
        obs = next_obs

    # Reduce exploration rate (agent becomes less random over time)
    agent.decay_epsilon()

  0%|          | 0/100000 [00:00<?, ?it/s]


TypeError: unhashable type: 'dict'